In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

In [ ]:
!pip install joblib

In [ ]:
!pip install scikit-learn-intelex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 MB 5.8 MB/s eta 0:00:00


In [ ]:
# plugin to sklearn for supporting gpu and leveraging gpu while training
from sklearnex import patch_sklearn
patch_sklearn()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
# from google.colab import userdata
# userdata.get('secretName')
project_dir = '/content/drive/MyDrive/mrunals_project'
# project_dir = r'D:\dev\shriekdj\mrunal_related_project_for_workload_data'

In [ ]:
zip_path = os.path.join(project_dir, 'converted_to_csv_again.zip')
zip_path

'/content/drive/MyDrive/mrunals_project/converted_to_csv_again.zip'

In [ ]:
import pandas as pd
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
from typing import Literal

# Function to load all CSVs from a zip into a single DataFrame without needing to extract non nesessory columns
def get_df(file, model_type: str):
  df = pd.read_csv(file)
  columns = list(df.columns)
  columns_only = list()
  columns_to_check = list()
  match model_type:
    case 'resource':
      columns_to_check = [
          'Submit Time',
          'Average CPU Time Used',
          'Requested Number of Processors',
          'Requested Time',
          # 'User ID',
          # 'Group ID',
          'Allocated Processors'
      ]
    case 'wait':
      columns_to_check = [
          'Allocated Processors',
          'Requested Number of Processors',
          'Submit Time',
          # 'User ID',
          'Wait Time'
      ]
    case 'energy':
      columns_to_check = [
          'Requested Number of Processors',
          'Allocated Processors',
          'Requested Memory',
          'Used Memory',
          'Average CPU Time Used',
          'Run Time'
      ]
    case _:
      raise Exception('Invalid Model Type')

  for col in columns_to_check:
    if col in columns:
      columns_only.append(col)

  return df[columns_only]

def load_data_from_zip(zip_path: str, model_type: str):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # List all file names in the zip that end with .csv
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        # Load each CSV file directly from the zip and concatenate into one DataFrame
        df_list = [get_df(z.open(file), model_type) for file in csv_files]
        full_df = pd.concat(df_list, ignore_index=True)


        full_df.dropna(
            # subset=['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID', 'Allocated Processors'],
            inplace=True
          )
        if model_type == 'energy':
          # Feature engineering: Creating the necessary features #Leaked Columns
          # full_df['Requested vs. Allocated Processors'] = full_df['Requested Number of Processors'] / full_df['Allocated Processors']
          # full_df['Requested vs. Allocated Memory'] = full_df['Requested Memory'] / full_df['Used Memory']
          full_df['CPU Utilization'] = full_df['Average CPU Time Used'] / full_df['Run Time'] #Run Time = Program/Job Run Time

          # Example energy consumption formula (you can adjust based on actual data):
          # Energy consumption is proportional to: (CPU Utilization) * (Allocated Processors) * (Used Memory) * (Run Time)
          # full_df['Energy Consumption (kWh)'] = full_df['CPU Utilization'] * full_df['Allocated Processors'] * full_df['Used Memory'] * full_df['Run Time'] / 1000  # in kWh
          full_df['Energy Consumption (kWh)'] = full_df['Average CPU Time Used'] * full_df['Allocated Processors'] * full_df['Used Memory'] / 1000  # in kWh
          full_df['Energy Consumption (kWh)'] = full_df['Energy Consumption (kWh)'].astype(float)
          # removed the Leaked Information
          full_df = full_df[
            [
              'Requested Number of Processors',
              'Allocated Processors',
              'Requested Memory',
              'Used Memory',
              'Average CPU Time Used',
              'Run Time',
              'Energy Consumption (kWh)'
            ]
          ]
        full_df.dropna(
            # subset=['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID', 'Allocated Processors'],
            how='any',
            inplace=True
        )
        return full_df

## Energy Consumption Optimization Model ( Graadient Boosting Regressor )

Predicts energy consumption based on system usage and job characteristics.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os, joblib

### This Section is for Training and Testing From Start

In [ ]:
from sklearn.ensemble import RandomForestRegressor

df_energy_cleaned = load_data_from_zip(zip_path, 'energy')

In [ ]:
df_energy_cleaned

,Requested Number of Processors,Allocated Processors,Requested Memory,Used Memory,Average CPU Time Used,Run Time,Energy Consumption (kWh)
57177765,0.0,1.000000,819200.0,0.0,2.0,1.544133e-05,0.000000
57177766,0.0,1.000000,819200.0,0.0,2.0,7.720666e-06,0.000000
57177767,0.0,1.000000,819200.0,0.0,2.0,7.720666e-06,0.000000
57177768,0.0,1.000000,819200.0,23208.0,2.0,1.254608e-05,46.416000
57177771,0.0,1.000000,819200.0,0.0,0.0,9.650833e-07,0.000000
...,...,...,...,...,...,...,...
67907070,0.0,0.001953,327680000.0,0.0,3.0,5.509292e-07,0.000000
67907071,0.0,0.001953,327680000.0,470436.0,4.0,8.263938e-07,3.675281
67907072,0.0,0.001953,327680000.0,472488.0,79.0,8.172116e-06,72.903422
67907073,0.0,0.001953,327680000.0,472488.0,353.0,3.434125e-05,325.758328


In [ ]:
# Proceed with your existing feature selection and model training steps
X_energy = df_energy_cleaned[
    [
        'Requested Number of Processors',
        'Allocated Processors',
        'Requested Memory',
        'Used Memory',
        'Average CPU Time Used',
        'Run Time',
        # 'User ID',
    ]
]
y_energy = df_energy_cleaned['Energy Consumption (kWh)'].astype(float)

In [ ]:
del df_energy_cleaned

In [ ]:
# Split the data
X_train_energy, X_test_energy, y_train_energy, y_test_energy = train_test_split(
    X_energy, y_energy, test_size=0.2, random_state=42
)

In [ ]:
# removing the variable from memory which are not in use from now
del X_energy
del y_energy

In [ ]:
y_train_energy

,Energy Consumption (kWh)
1024305,343.965078
3399489,2.956266
1141665,567.024719
4793150,0.000000
5464680,0.000000
...,...
7204212,0.000000
2234489,105.806250
4304572,189.536484
6550634,0.000000


In [ ]:
y_test_energy

,Energy Consumption (kWh)
6967877,19.897875
3844032,0.000000
3769615,286.949766
3861198,301.330859
3386746,0.000000
...,...
5672465,0.000000
2528390,0.000000
6071799,52504.266094
8124645,2.872500


In [ ]:
# Backing up test and training data ( It Not Saved on Google Drive But this Colab Files Navigation Icon)
joblib.dump(X_train_energy, 'X_train_energy.joblib')
joblib.dump(X_test_energy, 'X_test_energy.joblib')
joblib.dump(y_train_energy, 'y_train_energy.joblib')
joblib.dump(y_test_energy, 'y_test_energy.joblib')

['y_test_energy.joblib']

In [ ]:
# saving Them on Google Drive Within Zip File
with zipfile.ZipFile(
      os.path.join(
          project_dir,
          'model_energy_training_and_test_files.zip'
      ),
      'w',
      zipfile.ZIP_DEFLATED
    ) as zip:
    # Writing each file one by one
    for file in [
        'X_train_energy.joblib',
        'X_test_energy.joblib',
        'y_train_energy.joblib',
        'y_test_energy.joblib'
    ]:
        print(file)
        zip.write(os.path.join(os.getcwd(), file), arcname=file)

X_train_energy.joblib
X_test_energy.joblib
y_train_energy.joblib
y_test_energy.joblib


### This Section is for Loading Training and Testing Backup

In [ ]:
import joblib
# Restoring Backed Up Models
with zipfile.ZipFile(
      os.path.join(
        project_dir, 'model_energy_training_and_test_files.zip'
      ),
      'r'
    ) as z:
    # Load each CSV file directly from the zip and concatenate into one DataFrame
    X_train_energy = joblib.load(
        z.open('X_train_energy.joblib')
    )
    X_test_energy = joblib.load(
        z.open('X_test_energy.joblib')
    )
    y_train_energy = joblib.load(
        z.open('y_train_energy.joblib')
    )
    y_test_energy = joblib.load(
        z.open('y_test_energy.joblib')
    )
    # full_df = pd.concat(df_list, ignore_index=True)

### Section for Training The Models

In [ ]:
# Initialize and train the model with cpu
# model_energy = RandomForestRegressor(n_estimators=100, random_state=42)
model_energy = GradientBoostingRegressor(random_state=42, max_depth=5, n_estimators=700, learning_rate=0.05)
# model_energy = RandomForestRegressor(n_estimators=100, random_state=42
model_energy.fit(X_train_energy, y_train_energy)

GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=700,
                          random_state=42)

In [ ]:
# Initialize the model
gbr = GradientBoostingRegressor(random_state=42)

# Define parameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gbr,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Fit the grid search to the data
grid_search.fit(X_train_energy, y_train_energy)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test_energy)
rmse = np.sqrt(mean_squared_error(y_test_energy, y_pred))
print("RMSE of Best Model:", rmse)


Fitting 3 folds for each of 216 candidates, totalling 648 fits


In [ ]:
del X_train_energy
del y_train_energy

In [ ]:
# Predict and evaluate the model
predictions_energy = model_energy.predict(X_test_energy)
mse_energy = mean_squared_error(y_test_energy, predictions_energy)
print('RMSE for Energy Requirement Model:', np.sqrt(mse_energy))

RMSE for Energy Requirement Model: 991563.3637378945


In [ ]:
mean_y_test = y_test_energy.mean()
mean_y_test

88669.70528323945

In [ ]:
# @title Default title text
# saving model to google Drive as joblib file to use in future
import joblib
joblib.dump(
    model_energy,
    os.path.join(project_dir, 'model_energy_2.joblib')
)

['/content/drive/MyDrive/mrunals_project/model_energy_2.joblib']